In [1]:
import os
import warnings
import Codes.master_property_assessment as mpa
warnings.filterwarnings('ignore')

# List all .yml files in the current directory
yml_files = [file for file in os.listdir('.') if file.endswith('.yml')]

# Run property assessment on each .yml file
for yml_file in yml_files:
    mpa.run_property_assessment(yml_file)



Initiating property assessment for: 317 Newell St, Barberton, OH 44203
Property assessment for 317 Newell St, Barberton, OH 44203 completed in time (mins:secs): 0:14


In [4]:
import os
import glob
from datetime import datetime
import markdown
import re
import pdfkit

# Function to create CSS for HTML conversion
def create_css(output_css_path=None, font_family='Arial', font_size_base='14px', header_color='#333', primary_color='#555'):
    """
    Creates a CSS file to style HTML content for a report.

    Parameters:
    logo_path (str): Path to the company logo.
    output_css_path (str, optional): Path to save the generated CSS file. If not provided, it will be saved in the same directory as the logo.
    font_family (str): Font family to be used for the document text. Default is 'Arial'.
    font_size_base (str): Base font size for the document text. Default is '14px'.
    header_color (str): Color for the header text (h1, h2, h3). Default is '#333'.
    primary_color (str): Primary text color for the body. Default is '#555'.

    Returns:
    str: The path to the generated CSS file.
    """
    css_content = f'''
    body {{
        font-family: {font_family}, sans-serif;
        font-size: {font_size_base};
        color: {primary_color};
    }}
    .cover {{
        text-align: center;
        margin-top: 100px;
    }}
    .logo {{
        position: absolute;
        top: 10px;
        left: 10px;
        width: 100px;
    }}
    .footer {{
        position: fixed;
        bottom: 0;
        width: 100%;
        text-align: center;
        font-size: 12px;
    }}
    h1 {{
        font-size: 28px;
        font-weight: bold;
        color: {header_color};
    }}
    h2 {{
        font-size: 24px;
        font-weight: bold;
        color: {header_color};
    }}
    h3 {{
        font-size: 20px;
        font-weight: bold;
        color: {header_color};
    }}
    table {{
        width: 100%;
        border-collapse: collapse;
        margin-top: 20px;
    }}
    table, th, td {{
        border: 1px solid black;
    }}
    th, td {{
        padding: 10px;
        text-align: left;
    }}
    '''
    if output_css_path:
        if os.path.isdir(output_css_path):
            css_path = os.path.join(output_css_path, 'style.css')
        else:
            css_path = output_css_path
    with open(css_path, 'w') as css_file:
        css_file.write(css_content)
    return css_path

In [18]:
def convert_md_to_html(md_folder, html_folder, css_path):
    """
    Converts all Markdown files in the specified folder to HTML files, applying the provided CSS.

    Parameters:
    md_folder (str): Path to the folder containing Markdown files.
    html_folder (str): Path to the folder where HTML files will be saved.
    css_path (str): Path to the CSS file to be applied to the HTML files.
    """
    if not os.path.exists(html_folder):
        os.makedirs(html_folder)

    md_files = glob.glob(os.path.join(md_folder, '*.md'))

    for md_file in md_files:
        with open(md_file, 'r') as file:
            md_content = file.read()

            # Preprocess Markdown content to handle image paths
            def replace_image_paths(match):
                alt_text = match.group(1)
                img_path = match.group(2).replace('\\', '/')
                if not os.path.isabs(img_path):
                    img_path = os.path.relpath(os.path.join(md_folder, img_path), html_folder).replace('\\', '/')
                return f'<img src="{img_path}" alt="{alt_text}" style="width: 80%; margin-top: 20px;">'

            md_content = re.sub(r'!\[([^\]]+)\]\(([^\)]+)\)', replace_image_paths, md_content)

            # Convert Markdown to HTML
            html_content = markdown.markdown(md_content, extensions=['tables'])

            # Create full HTML with CSS link
            html_output = f"""
            <html>
            <head>
                <link rel="stylesheet" type="text/css" href="file:///{css_path}">
            </head>
            <body>
                {html_content}
            </body>
            </html>
            """

            # Save HTML to file
            html_file_name = os.path.splitext(os.path.basename(md_file))[0] + '.html'
            html_file_path = os.path.join(html_folder, html_file_name)
            with open(html_file_path, 'w') as html_file:
                html_file.write(html_output)

    print(f"Converted {len(md_files)} Markdown files to HTML in folder: {html_folder}")


In [20]:
def convert_html_to_pdf(html_folder, output_pdf_path):
    """
    Converts all HTML files in the specified folder into a single PDF file.

    Parameters:
    html_folder (str): Path to the folder containing HTML files.
    output_pdf_path (str): Path to save the generated PDF file.
    """
    html_files = glob.glob(os.path.join(html_folder, '*.html'))

    if not html_files:
        print("No HTML files found to convert.")
        return

    try:
        pdfkit.from_file(html_files, output_pdf_path)
        print(f"PDF successfully generated at {output_pdf_path}")
    except Exception as e:
        print(f"An error occurred while generating the PDF: {e}")

In [6]:
csspath = create_css(output_css_path="Utilities/", font_family='Arial', font_size_base='14px', header_color='#333', primary_color='#555')

In [19]:
convert_md_to_html("317 Newell St/Documentation/MD Files/", "317 Newell St/Documentation/HTML Files/", csspath)

Converted 6 Markdown files to HTML in folder: 317 Newell St/Documentation/HTML Files/


In [22]:
convert_html_to_pdf("317 Newell St/Documentation/HTML Files/", "317 Newell St/Documentation/PDF Files/")

An error occurred while generating the PDF: wkhtmltopdf reported an error:
Exit with code 1 due to network error: ProtocolUnknownError

